# Proyecto final

Juan Pablo Rodas 23007521

Dataset empleado para el proyecto: [Wine Rating & Price](https://www.kaggle.com/datasets/budnyak/wine-rating-and-price?select=Sparkling.csv)

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Carga de datos

In [2]:
red_data = pd.read_csv('./data/Red.csv')
rose_data = pd.read_csv('./data/Rose.csv')
white_data = pd.read_csv('./data/White.csv')

In [3]:
red_data.describe()

,Rating,NumberOfRatings,Price
count,8666.000000,8666.000000,8666.000000
mean,3.890342,415.287445,39.145065
std,0.308548,899.726373,84.936307
min,2.500000,25.000000,3.550000
25%,3.700000,66.000000,10.680000
50%,3.900000,157.000000,18.200000
75%,4.100000,401.000000,38.142500
max,4.800000,20293.000000,3410.790000


In [4]:
rose_data.describe()

,Rating,NumberOfRatings,Price
count,397.000000,397.000000,397.000000
mean,3.741310,261.727960,12.525743
std,0.273042,1568.108597,16.037644
min,2.700000,25.000000,3.670000
25%,3.600000,40.000000,7.190000
50%,3.700000,72.000000,8.900000
75%,3.900000,167.000000,12.850000
max,4.800000,29531.000000,249.000000


In [5]:
white_data.describe()

,Rating,NumberOfRatings,Price
count,3764.000000,3764.000000,3764.000000
mean,3.817906,187.567216,20.620319
std,0.266827,1071.470441,30.928415
min,2.200000,25.000000,3.740000
25%,3.600000,43.000000,9.260000
50%,3.800000,77.000000,13.150000
75%,4.000000,174.250000,20.865000
max,4.900000,62980.000000,681.370000


## Limpieza y procesamiento

In [6]:
red_data['Type'] = 'Red'
rose_data['Type'] = 'Rose'
white_data['Type'] = 'White'

In [7]:
complete_df = pd.concat([red_data,rose_data,white_data], ignore_index = True)

In [8]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12827 entries, 0 to 12826
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             12827 non-null  object 
 1   Country          12827 non-null  object 
 2   Region           12827 non-null  object 
 3   Winery           12827 non-null  object 
 4   Rating           12827 non-null  float64
 5   NumberOfRatings  12827 non-null  int64  
 6   Price            12827 non-null  float64
 7   Year             12827 non-null  object 
 8   Type             12827 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 902.0+ KB


In [9]:
complete_df.isna().sum()

Name               0
Country            0
Region             0
Winery             0
Rating             0
NumberOfRatings    0
Price              0
Year               0
Type               0
dtype: int64

## Quitar caracteres especiales

In [10]:
complete_df['Name'] = complete_df['Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
complete_df['Winery'] = complete_df['Winery'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [11]:
complete_df.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year,Type
0,Pomerol 2011,France,Pomerol,Chateau La Providence,4.2,100,95.00,2011,Red
1,Lirac 2017,France,Lirac,Chateau Mont-Redon,4.3,100,15.50,2017,Red
2,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015,Red
3,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019,Red
4,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016,Red
